In [8]:
import numpy as np
import random
import optuna
import pandas as pd
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, balanced_accuracy_score
import json
import os
from optuna.visualization import plot_optimization_history
import time
from datetime import datetime
import joblib

In [9]:
SEED = 1234
 # Se puede cambiar a "precision" o "recall" o "f1-score"
SCORE = "f1-score"
model_name = 'HistGradientBoosting'

In [10]:
def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)

In [11]:
def get_trend_changes_report_dict(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(
        y_df["is_changed_trend_test"][:-1], 
        y_df["is_changed_trend_predict"][:-1], 
        digits=4,
        output_dict=True,
        zero_division=0
    )

def trend_changes_score(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(y_df["is_changed_trend_test"][:-1], y_df["is_changed_trend_predict"][:-1], digits=4)

def trend_changes_true(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    report = classification_report(
        y_df["is_changed_trend_test"][:-1],
        y_df["is_changed_trend_predict"][:-1],
        output_dict=True,
        zero_division=0
    )
    return report["True"][SCORE]

In [12]:
# Cargar datos
train = pd.read_csv("../../../data/post_cleaning/training_set.csv", parse_dates=["date"])
val = pd.read_csv("../../../data/post_cleaning/validation_set.csv", parse_dates=["date"])
X_train = train.drop(columns=["date", "target_trend"]).values
y_train = train["target_trend"].values
X_val = val.drop(columns=["date", "target_trend"]).values
y_val = val["target_trend"].values

In [13]:
def objective(trial):
    set_seeds(SEED)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.2, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 16)
    max_iter = trial.suggest_int("max_iter", 100, 500)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
    l2_regularization = trial.suggest_float("l2_regularization", 0.0, 2.0)
    max_bins = trial.suggest_int("max_bins", 128, 255)

    model = HistGradientBoostingClassifier(
        learning_rate=learning_rate,
        max_depth=max_depth,
        max_iter=max_iter,
        min_samples_leaf=min_samples_leaf,
        l2_regularization=l2_regularization,
        max_bins=max_bins,
        random_state=SEED
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    score = trend_changes_true(y_val, y_pred)
    return score

In [14]:
# Crear estudio y medir tiempo de Optuna
t0 = time.perf_counter()

study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
)
study.optimize(objective, n_trials=500)

opt_duration_sec = time.perf_counter() - t0
n_trials_run = len(study.trials)

print(f"Optuna duró {opt_duration_sec:.2f}s en {n_trials_run} trials")

[I 2025-09-16 22:36:26,139] A new study created in memory with name: no-name-f02e573d-45b7-4b5a-a516-483b247b51a8
[I 2025-09-16 22:36:28,687] Trial 0 finished with value: 0.5074626865671642 and parameters: {'learning_rate': 0.01774894524799066, 'max_depth': 11, 'max_iter': 275, 'min_samples_leaf': 16, 'l2_regularization': 1.559951616237607, 'max_bins': 162}. Best is trial 0 with value: 0.5074626865671642.
[I 2025-09-16 22:36:33,624] Trial 1 finished with value: 0.5454545454545454 and parameters: {'learning_rate': 0.022892239910869177, 'max_depth': 14, 'max_iter': 484, 'min_samples_leaf': 18, 'l2_regularization': 0.7156345399157333, 'max_bins': 192}. Best is trial 1 with value: 0.5454545454545454.
[I 2025-09-16 22:36:36,151] Trial 2 finished with value: 0.5 and parameters: {'learning_rate': 0.07748288441159408, 'max_depth': 12, 'max_iter': 248, 'min_samples_leaf': 12, 'l2_regularization': 1.0061663306156194, 'max_bins': 129}. Best is trial 1 with value: 0.5454545454545454.
[I 2025-09-16

Optuna duró 491.57s en 500 trials


In [15]:
# Visualizar el historial de optimización
plot_optimization_history(study)

In [16]:
print("Mejores hiperparámetros encontrados:")
print(study.best_params)
print(f"Mejor score de trend change: {study.best_value:.4f}")
best_params = study.best_params

Mejores hiperparámetros encontrados:
{'learning_rate': 0.1575206128209753, 'max_depth': 3, 'max_iter': 304, 'min_samples_leaf': 19, 'l2_regularization': 0.8279046766404491, 'max_bins': 128}
Mejor score de trend change: 0.6241


In [17]:
# Guardar los mejores hiperparámetros y su valor
history = []
if os.path.exists("best_hyperparams.json"):
    try:
        with open("best_hyperparams.json", "r") as f:
            history = json.load(f)
    except (json.JSONDecodeError, ValueError):
        history = []

# Guardar ambos en un solo diccionario
history.append({
    "params": study.best_params,
    "value": study.best_value
})
# Guardar el historial de hiperparámetros
with open("best_hyperparams.json", "w") as f:
    json.dump(history, f, indent=2)

In [18]:
# Cargar historial de hiperparámetros y valores
import json

with open("best_hyperparams.json", "r") as f:
    history = json.load(f)

# Escoger el último (más reciente)
best_params = history[-1]["params"]
best_value = history[-1]["value"]

# Para ver todos:
for i, entry in enumerate(history):
    print(f"Hiperparámetros #{i+1}: {entry['params']}, Valor: {entry['value']}")

# Para escoger uno específico (por índice):
# best_params = history[indice_que_quieras]["params"]
# best_value = history[indice_que_quieras]["value"]

Hiperparámetros #1: {'learning_rate': 0.1575206128209753, 'max_depth': 3, 'max_iter': 304, 'min_samples_leaf': 19, 'l2_regularization': 0.8279046766404491, 'max_bins': 128}, Valor: 0.624113475177305


In [19]:
# Entrenar modelo final con los mejores hiperparámetros
set_seeds(SEED)
final_model = HistGradientBoostingClassifier(
    learning_rate=best_params["learning_rate"],
    max_depth=best_params["max_depth"],
    max_iter=best_params["max_iter"],
    min_samples_leaf=best_params["min_samples_leaf"],
    l2_regularization=best_params["l2_regularization"],
    max_bins=best_params["max_bins"],
    random_state=SEED
)
final_model.fit(X_train, y_train)
joblib.dump(final_model, "HistGradientBoosting.joblib")

['HistGradientBoosting.joblib']

In [22]:
# Predicciones con el modelo final
y_val_pred = final_model.predict(X_val)
print("\nHistGradientBoostingClassifier Trend Changes Score:\n", trend_changes_score(y_val, y_val_pred))


HistGradientBoostingClassifier Trend Changes Score:
               precision    recall  f1-score   support

       False     0.8673    0.8950    0.8809       219
        True     0.6567    0.5946    0.6241        74

    accuracy                         0.8191       293
   macro avg     0.7620    0.7448    0.7525       293
weighted avg     0.8141    0.8191    0.8160       293



In [23]:
# Reporte completo: precisión, recall y F1 por clase
report = classification_report(y_val, y_val_pred, digits=4)
print("Gradient Boosting Classifier Report:\n", report)
print("Balanced accuracy:", balanced_accuracy_score(y_val, y_val_pred))

Gradient Boosting Classifier Report:
               precision    recall  f1-score   support

          -1     0.7843    0.8696    0.8247        92
           0     0.4231    0.2245    0.2933        49
           1     0.8072    0.8758    0.8401       153

    accuracy                         0.7653       294
   macro avg     0.6715    0.6566    0.6527       294
weighted avg     0.7360    0.7653    0.7442       294

Balanced accuracy: 0.656624002257908


In [24]:
# --- Exportar y Comparar Métricas de Modelos (Validación) ---

# 1. Definir el nombre del modelo actual y el archivo de salida
output_file = '../../../score_models/model_comparison_metrics.csv'

# 2. Calcular el reporte de clasificación estándar
report_dict = classification_report(y_val, y_val_pred, output_dict=True, zero_division=0)
precision = report_dict['macro avg']['precision']
recall = report_dict['macro avg']['recall']
f1_score = report_dict['macro avg']['f1-score']


# 3. Calcular el reporte de cambio de tendencia
report = get_trend_changes_report_dict(y_val, y_val_pred)
trend_change_precision = report['True']['precision']
trend_change_recall = report['True']['recall']
trend_change_f1_score = report['True']['f1-score']

# 4. Organizar las nuevas métricas
new_metrics = {
    'precision': precision,
    'recall': recall,
    'f1_score': f1_score,
    'trend_change_precision': trend_change_precision,
    'trend_change_recall': trend_change_recall,
    'trend_change_f1_score': trend_change_f1_score,
    "optuna_duration_sec": opt_duration_sec,
    "n_trials": n_trials_run
}

# 5. Cargar, actualizar y guardar el DataFrame de comparación
try:
    # Intentar cargar el archivo existente
    comparison_df = pd.read_csv(output_file, index_col='model')
    # Si existe, actualizar o añadir la fila para el modelo actual
    comparison_df.loc[model_name] = new_metrics
except FileNotFoundError:
    # Si no existe, crear un DataFrame nuevo directamente con los datos actuales
    comparison_df = pd.DataFrame([new_metrics], index=[model_name])

# Guardar el DataFrame actualizado en el CSV
comparison_df.to_csv(output_file, index_label='model')